In [2]:
from comdb2 import dbapi2
import pandas as pd 
from datetime import datetime, date
from tqdm.auto import tqdm

In [3]:
db_name = "nanadb"
connection = dbapi2.connect(db_name, autocommit=True, tier="dev")

In [66]:
# Setting an appropriate SQL query
num_rows_to_fetch = 500
offset = 0
fetched_dfs = []
date_cutoff = date(2021, 6, 1)

sql_query_big = f"""
    with r1 as (
            SELECT * from release 
             WHERE wire_id > 0 and wire_id NOT IN (25,2345,96,3543,584,474,1719,3447,586,587,97,2640)
                AND CAST(toa AS DATE) > CAST('2023-01-01' AS DATE)
    ),
    story_nicode_grouped as (
        SELECT suid, group_concat(nicode) as nicode from story_nicode group by suid
    ),
    story_ticker_grouped as (
        select suid, group_concat(ticker) as ticker from story_ticker group by suid
    ),
    release_nicode_grouped as (
        SELECT release_id, group_concat(nicode) as nicode from release_nicode group by release_id
    ),
    release_ticker_grouped as (
        SELECT release_id, group_concat(ticker) as ticker from release_ticker group by release_id
    ),
    bullet_grouped as (
        SELECT release_id, group_concat(text, '|||') as text from bullet group by release_id
    )
        SELECT
            S.suid AS story_suid,
            R.suid AS release_suid,
            R.bundle_id AS release_bundle_id,
            S.release_id AS release_id,
            ST.name AS story_type,
            RT.name AS release_type,
            S.toa AS story_toa,
            R.toa AS release_toa,
            S.headline AS story_headline,
            S.wire AS story_wire,
            S.wire_class AS story_wire_class,
            R.wire_id AS release_wire,
            R.wire_class AS release_wire_class,
            R.web_url AS release_web_url,
            R.subject AS release_subject, 
            R.body AS release_body,
            B.text AS bullet,
            SNC.nicode AS story_nicode,
            STR.ticker AS story_ticker,
            RNC.nicode AS release_nicode,
            RTR.ticker AS release_ticker
            FROM story S
            JOIN r1 R ON S.release_id = R.id
            JOIN release_type RT ON R.release_type = RT.release_type
            JOIN story_type ST ON S.story_type = ST.story_type
            JOIN release_nicode_grouped RNC ON R.id = RNC.release_id
            JOIN story_nicode_grouped SNC ON S.suid = SNC.suid
            JOIN release_ticker_grouped RTR ON R.id = RTR.release_id
            JOIN story_ticker_grouped STR ON S.suid = STR.suid
            JOIN bullet_grouped B ON B.release_id = R.id
            WHERE S.language = 'en'
            AND R.language = 'en'
"""

sql_query_no_nicodes = f"""
    with r1 as (
            SELECT * from release 
             WHERE wire_id > 0 and wire_id NOT IN (25,2345,96,3543,584,474,1719,3447,586,587,97,2640)
                AND CAST(toa AS DATE) > CAST('2023-01-01' AS DATE)
    )
        SELECT 
            S.suid AS story_suid,
            R.suid AS release_suid,
            R.bundle_id AS release_bundle_id,
            S.release_id AS release_id,
            ST.name AS story_type,
            RT.name AS release_type,
            S.toa AS story_toa,
            R.toa AS release_toa,
            S.headline AS story_headline,
            S.wire AS story_wire,
            S.wire_class AS story_wire_class,
            R.wire_id AS release_wire,
            R.wire_class AS release_wire_class,
            R.web_url AS release_web_url,
            R.subject AS release_subject, 
            R.body AS release_body
            FROM story S
            JOIN r1 R ON S.release_id = R.id
            JOIN release_type RT ON R.release_type = RT.release_type
            JOIN story_type ST ON S.story_type = ST.story_type
            WHERE S.language = 'en'
            AND R.language = 'en'
"""
    
for idx, df in enumerate(
    pd.read_sql(sql_query_no_nicodes, con=connection, chunksize=num_rows_to_fetch)
):
    df.to_csv(f'one-fetch-{idx}.csv')
    if idx % 5 == 0:
        print(f'fetched {idx} rows...')

fetched 0 rows...
fetched 5 rows...
fetched 10 rows...
fetched 15 rows...
fetched 20 rows...
fetched 25 rows...
fetched 30 rows...
fetched 35 rows...
fetched 40 rows...


In [67]:
from unidecode import unidecode

In [68]:
import glob
from bs4 import BeautifulSoup
import re
from tqdm.auto import tqdm
tqdm.pandas()

fetched_files = glob.glob('one-fetch-*.csv')
fetched_files = sorted(fetched_files, key=lambda x: int(re.search('-(\d+).csv', x)[1]))

dfs_sans_body = []
body_dfs = []
dump_every = 5
proc_num = 0
for i, f in tqdm(enumerate(fetched_files), total=len(fetched_files)):
    df = pd.read_csv(f, index_col=0)
    df_proc = df.loc[lambda df: df['release_body'].fillna('').str.strip() != '']
    
    bodies = df_proc[['release_suid', 'release_body']].drop_duplicates()
    bodies['processed_release_body'] = (
        bodies['release_body']
            .apply(lambda x: BeautifulSoup(x).get_text().strip())
            .apply(unidecode)
            .drop(columns='release_body')
    )
    df_proc = df_proc.drop(columns='release_body')
    dfs_sans_body.append(df_proc)
    body_dfs.append(bodies)
    
    if i % dump_every == 0:
        pd.concat(dfs_sans_body).to_csv(f'processed-nana-{proc_num}.csv')
        pd.concat(body_dfs).to_csv(f'processed-nana-bodies-{proc_num}.csv')        
        proc_num += 1
        dfs_sans_body = []
        body_dfs = []

100%|██████████| 42/42 [11:17<00:00, 16.13s/it]


In [69]:
processed_files = glob.glob('processed-nana-*.csv')

In [70]:
list(filter(lambda x: 'bodies' not in x, processed_files))

['processed-nana-7.csv',
 'processed-nana-2.csv',
 'processed-nana-6.csv',
 'processed-nana-3.csv',
 'processed-nana-5.csv',
 'processed-nana-0.csv',
 'processed-nana-1.csv',
 'processed-nana-8.csv',
 'processed-nana-4.csv']

In [77]:
processed_files = glob.glob('processed-nana-*.csv')
processed_files = sorted(processed_files, key=lambda x: int(re.search('-(\d+).csv', x)[1]))

all_processed_dfs = []
all_processed_body_dfs = [] 
cutoff = 25
num_combined = num_bodies_combined = 0
for idx, p_filename in tqdm(enumerate(processed_files)):
    df = pd.read_csv(p_filename, index_col=0)
    if 'bodies' in p_filename:
        all_processed_body_dfs.append(df)
    else:
        all_processed_dfs.append(df)
    if int(idx / 2)  % cutoff == 0:
        if len(all_processed_dfs) > 1:
            pd.concat(all_processed_dfs).to_csv(
                f'big-processed-nana-file-{num_combined}.csv.gz', compression='gzip'
            )
            num_combined +=1
            all_processed_dfs = []
        
        if len(all_processed_body_dfs) > 1:
            pd.concat(all_processed_body_dfs).to_csv(
                f'big-processed-nana-file-bodies-{num_bodies_combined}.csv.gz', compression='gzip'
            )
            num_bodies_combined +=1
            all_processed_body_dfs = []

            
pd.concat(all_processed_dfs).to_csv(
    f'big-processed-nana-file-{num_combined}.csv.gz', compression='gzip'
)
pd.concat(all_processed_body_dfs).to_csv(
    f'big-processed-nana-file-bodies-{num_combined}.csv.gz', compression='gzip'
)

18it [00:11,  1.53it/s]


In [72]:
full_processed_df = pd.concat(all_processed_dfs)

# Wrap data from Comdb2

In [7]:
import glob
import pandas as pd 

In [8]:
suid_rid_map_files = glob.glob('../data/zomo-downloads/*suids-rids*')
suid_rid_maps = pd.concat(list(map(pd.read_excel, suid_rid_map_files)))`

In [34]:
suid_rid_maps.to_csv('../data/zomo-downloads/all_suid_rid_maps.csv')

In [12]:
from tqdm.auto import tqdm

In [15]:
release_body_files = glob.glob('../data/zomo-downloads/*release*')
release_bodies = [] 

for f in tqdm(release_body_files):
    try:
        release_bodies.append(pd.read_excel(f))
    except:
        print(f'failed on {f}...')

  0%|          | 0/23 [00:00<?, ?it/s]

failed on ../data/zomo-downloads/~$2023-01-01__2023-02-01__release-data.xlsx...


In [17]:
release_bodies_df = pd.concat(release_bodies)